**Name: Kavya Chigurupati**

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [34]:
# CORA='/content/gdrive/MyDrive/CORA/'
CORA ='/content/gdrive/MyDrive/ITCS-6156-ML/CORA/'

with open(CORA + 'vocab-lower.pickle', 'rb') as f:
    #vocab dic is list of all words in the dictionery
    vocab_dic = list(map(lambda x: x.decode('utf-8'), pickle.load(f, encoding='utf-8')))
    # convert all the data into lower case

# assign token id to each word and add it to pickle-kc
with open(CORA + 'vocab_labels.pickle', 'rb') as f:
    #The list of possible labels 
    label_dic = list(map(lambda x: x.decode('utf-8'), pickle.load(f, encoding='bytes')))

# remove tokens that occur only once - but here since it is citation and may have names we didnot do it
    
with open(CORA + 'trained_embedding.pickle', 'rb') as f:
    #The trained embedding on Wikipedia using GLOVE algorithm 
    #Each column is the embedding of a word (with respect to the vocab dictionary) 
    embedding = pickle.load(f, encoding='bytes')
    vocabulary_size = 20608
    embedding_size = 100


with open(CORA + 'xdata-lower.pickle', 'rb') as f:
    #Each row is a citation. Each cell is the token id (with respect to the vocab dictionary) within the citation.
    xdata = pickle.load(f, encoding='bytes')

with open(CORA + 'Y_train.pickle', 'rb') as f:
    ydata = pickle.load(f, encoding='bytes')

with open(CORA + 'xval-lower.pickle', 'rb') as f:
    xval = pickle.load(f, encoding='bytes')

with open(CORA + 'Y_dev.pickle', 'rb') as f:
    yval = pickle.load(f, encoding='bytes')

with open(CORA + 'xtest-lower.pickle', 'rb') as f:
    xtest = pickle.load(f, encoding='bytes')

with open(CORA + 'Y_test.pickle', 'rb') as f:
    ytest = pickle.load(f, encoding='bytes')





In [35]:
len(vocab_dic)

18049

In [36]:
vocab_dic[1:10]

['.', ',', '-', 'and', 'of', 'in', '(', ')', 'a']

In [37]:
label_dic[5:6]

['author']

In [38]:
print(len(xdata[0]))
print(xdata[0])

118
[    9     1  5259     2    22     1  8920     2     4    41     1     3
    26     1   161  3309     1 16305  6634   277  1933   970 11234  2751
     1     6    25     1    40     1   709     2    22     1    25     1
 11654     2     4    38     1  7646     2   103     2    29     1   357
     1  5710     3  9418  1846    69     2    93     1     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0]


In [39]:
ydata[0]

array([ 5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        3,  3,  3,  3,  3,  3,  3,  3, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,  4,  4,  4,  4,  4,  4,
        4,  4,  4,  4, 11, 11,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])

In [40]:

print("vocab size:", len(vocab_dic))
print(vocab_dic[100], vocab_dic[200])

vocab size: 18049
no formal


In [41]:
def cosine_similarity(v1, v2):
  norm_v1 = np.linalg.norm(v1)
  norm_v2 = np.linalg.norm(v2)
  return np.dot(v1,v2)/(norm_v1*norm_v2)

In [42]:
v1_id = vocab_dic.index('positive')
v2_id = vocab_dic.index('constructive')
v3_id = vocab_dic.index('conference')

v1_vec = embedding[v1_id]
v2_vec = embedding[v2_id]
v3_vec = embedding[v3_id]


In [43]:
print(v1_id)
print(v1_vec)
print(len(v1_vec))
print(len(v2_vec))

4659
[ 0.54177    1.0436     0.55266   -0.62825    0.30594    0.28087
 -0.35377    0.35549    0.95647    0.50894   -0.26715    0.55767
 -1.2315     0.40356   -0.096982  -0.086239  -0.14343    0.17824
 -0.041343  -1.0227    -0.17425   -0.61138    0.8514     0.27748
  0.0053477 -0.6192    -0.84856   -0.51399   -0.49071   -0.43858
  0.88657   -0.50032   -0.54597   -0.072396  -0.55092    0.17349
  0.019984  -0.070796  -0.081909  -0.49314   -0.51248    0.50913
  0.3783     1.2291     0.20437    0.012887   0.32293   -0.21534
  0.5537     0.80678   -0.52994    0.46817   -0.81708   -0.018085
 -0.60689   -0.83048   -0.3863     0.22914    0.52167   -0.1351
  0.23068    0.20365    0.53207   -0.28598    0.32816   -0.28617
 -0.465      0.3136    -0.82072   -0.3596    -1.0122    -0.33755
 -0.10101   -0.13263   -1.0847    -0.6569    -0.45667   -0.52978
 -0.32365    1.0648    -0.53276    0.43342    0.61473   -0.79105
  0.24635   -0.677      0.68037   -0.79789   -0.27437    0.79626
  0.59855    0.07510

In [44]:
cosine_similarity(v1_vec,v2_vec)

0.40829688468111025

In [45]:
cosine_similarity(v1_vec, v3_vec)

0.226298426910638

In [46]:
#construct the sentence
print(xdata[1])
#each cell is the token id and 0 means padding.


[   16     1  4426     2    33     1  2014     2     4    38     1  8853
     3 14152     1   189     5  1901    37    59   839   128     4   319
   155     1   102   282    62     2    46     7    46     8     2   297
     1     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0]


In [47]:
def get_sentence(x):
  x_nopad = list(filter(lambda i: True if i > 0 else False, x))
  return ' '.join([vocab_dic[id] for id in x_nopad])

def get_label(y):
  y_nopad = list(filter(lambda i: True if i > 0 else False, y))
  return ' '.join([label_dic[id] for id in y_nopad])

def print_sentence_label_pair(x, y):
  x_nopad = list(filter(lambda i: True if i > 0 else False, x))
  y_nopad = list(filter(lambda i: True if i > 0 else False, y))
  for i in range(len(x_nopad)):
    print(label_dic[y_nopad[i]],'\t\t' ,vocab_dic[x_nopad[i]],)

def print_sentence_pred_label(x, y, yt):
  x_nopad = list(filter(lambda i: True if i > 0 else False, x))
  for i in range(len(x_nopad)):
    print(label_dic[yt[i]], label_dic[y[i]],'\t\t'  ,vocab_dic[x_nopad[i]],)


In [48]:
print(get_sentence(xdata[10]))


ramadge , p . , & wonham , w . ( 1989 ) . the control of discrete event systems . proceedings of the ieee , 77 ( 1 ) , 81 - 98 .


In [49]:
 print(get_label(ydata[10]))
 

author author author author author author author author author author date date date date title title title title title title title booktitle booktitle booktitle booktitle booktitle volume volume volume volume volume pages pages pages pages


In [50]:
print_sentence_label_pair(xdata[10], ydata[10])
# print(get_sentence(xdata[10]))
# print(get_label(ydata[10]))

author 		 ramadge
author 		 ,
author 		 p
author 		 .
author 		 ,
author 		 &
author 		 wonham
author 		 ,
author 		 w
author 		 .
date 		 (
date 		 1989
date 		 )
date 		 .
title 		 the
title 		 control
title 		 of
title 		 discrete
title 		 event
title 		 systems
title 		 .
booktitle 		 proceedings
booktitle 		 of
booktitle 		 the
booktitle 		 ieee
booktitle 		 ,
volume 		 77
volume 		 (
volume 		 1
volume 		 )
volume 		 ,
pages 		 81
pages 		 -
pages 		 98
pages 		 .


In [51]:
#calucate the hamming loss between gold data and predication
def token_level_loss(cpred, ctrue):
  try:
    label_num = np.shape(cpred)[1]
    pred = cpred.astype(np.int)
    true = ctrue.astype(np.int)
    n1 = len(pred)
    n2 = len(true)
    assert n1 == n2
    hloss = 0.0
    exact_acc = 0.0
    for i in range(n1):
      sample_loss = 0.0
      exact = 0.0
      xp = pred[i]
      xt = true[i]
      cnp = 0
      for j in range(label_num):
        if (xt[j] == 0):
          continue
        cnp+=1
        if xp[j] != xt[j]:
          sample_loss += 1.0
        else:
          exact +=1
      sample_loss = sample_loss / cnp
      exact = exact / cnp
      hloss += sample_loss
      exact_acc += exact
    return (hloss / n1, exact_acc / n1)
  except Warning:
    return (0.0,0.0);

def perf(ytr_pred, yval_pred, yts_pred, ydata, yval, ytest):
    global best_val
    global test_val

    hm_ts, ex_ts = token_level_loss(yts_pred, ytest)
    hm_tr, ex_tr = token_level_loss(ytr_pred, ydata)
    hm_val, ex_val = token_level_loss(yval_pred, yval)
    if ex_val > best_val:
        best_val = ex_val
        test_val = ex_ts
    return ("Train: %0.3f Val: %0.3f Test: %0.3f -- Best Val: %0.3f Test: %0.3f" % (ex_tr, ex_val, ex_ts, best_val, test_val))

**Model 1** : Bidirectional Lstm 

Accuracy: 53.3%

In [54]:
output_size = len(label_dic)
max_length_output = 118
class CitationNetwork(tf.keras.Model):
  def __init__(self, network_type = 'SimpleLSTM'):
      super(CitationNetwork, self).__init__()

      # self.optimizer = tf.keras.optimizers.RMSprop(1e-3)
      self.optimizer = tf.keras.optimizers.Adam(1e-3)
      self.embedding = layers.Embedding(input_dim=vocabulary_size, output_dim=embedding_size)
      
      self.forward_layer = layers.LSTM(32, return_sequences=True, return_state=True)
      self.backward_layer = layers.LSTM(32, return_sequences=True,return_state=True,go_backwards=True)
      self.rnn = layers.Bidirectional(self.forward_layer,merge_mode="concat",backward_layer = self.backward_layer)
      

      self.fc = layers.Dense(output_size)
      
 
  def call(self, x, predict=True):
      emb = self.embedding(x)
      output,_,_,_,_ = self.rnn(emb)
      

      output = self.fc(output)
      if predict:
        return tf.math.argmax(output, axis=-1)
      return output
  
  def get_loss(self, ylogits, yt):
    # y - one hot vector which makes it easy to compute loss 
    ylabels = tf.one_hot(ybatch, output_size)
    cross_ent = tf.nn.softmax_cross_entropy_with_logits(logits=ylogits, labels=ylabels);
    return tf.reduce_sum(cross_ent)

@tf.function
def train_step(xbatch, ybatch):
    loss = 0
    with tf.GradientTape() as tape:
        prediction = cite(xbatch, False)
        # prediction = cite(xbatch, False)
        loss = cite.get_loss(prediction, ybatch)
        
    gradients = tape.gradient(loss, cite.trainable_variables)
    cite.optimizer.apply_gradients(zip(gradients, cite.trainable_variables))
    return loss

cite = CitationNetwork();

        

In [55]:
train_size = xdata.shape[0]
batch_size = 10
train_dataset = (tf.data.Dataset.from_tensor_slices(np.hstack((xdata, ydata)))
                 .shuffle(train_size).batch(batch_size))
# print(xdata.shape)
num_epoch = 1000
i = 0;
best_val = 0
test_val = 0

while i < num_epoch:
  loss = 0;
  for batch in train_dataset:
    xbatch = batch[:, :max_length_output]
    ybatch = batch[:, max_length_output:]
    loss += train_step(xbatch, ybatch)

  ypred_test = cite(xtest, predict=True).numpy()
  ypred_val = cite(xval, predict=True).numpy()
  ypred_train = cite(xdata, predict=True).numpy()
  print(i, loss.numpy(), perf(ypred_train, ypred_val, ypred_test, ydata, yval, ytest) )
  i = i+1
  # break

0 65470.508 Train: 0.000 Val: 0.000 Test: 0.000 -- Best Val: 0.000 Test: 0.000
1 36676.49 Train: 0.245 Val: 0.248 Test: 0.273 -- Best Val: 0.248 Test: 0.273
2 32636.643 Train: 0.467 Val: 0.477 Test: 0.482 -- Best Val: 0.477 Test: 0.482
3 29189.953 Train: 0.445 Val: 0.463 Test: 0.470 -- Best Val: 0.477 Test: 0.482
4 26219.719 Train: 0.485 Val: 0.482 Test: 0.506 -- Best Val: 0.482 Test: 0.506
5 23634.148 Train: 0.467 Val: 0.490 Test: 0.533 -- Best Val: 0.490 Test: 0.533
6 22149.207 Train: 0.439 Val: 0.475 Test: 0.511 -- Best Val: 0.490 Test: 0.533
7 20935.277 Train: 0.423 Val: 0.458 Test: 0.496 -- Best Val: 0.490 Test: 0.533
8 20298.021 Train: 0.424 Val: 0.469 Test: 0.511 -- Best Val: 0.490 Test: 0.533
9 19996.432 Train: 0.412 Val: 0.456 Test: 0.493 -- Best Val: 0.490 Test: 0.533
10 19617.781 Train: 0.418 Val: 0.464 Test: 0.508 -- Best Val: 0.490 Test: 0.533
11 19204.236 Train: 0.413 Val: 0.455 Test: 0.493 -- Best Val: 0.490 Test: 0.533
12 19242.908 Train: 0.414 Val: 0.455 Test: 0.498 --

In [56]:
print_sentence_pred_label(xdata[1], ypred_train[1], ydata[1])

author author 		 m
author author 		 .
author author 		 kitsuregawa
author author 		 ,
author author 		 h
author author 		 .
author author 		 tanaka
author author 		 ,
author author 		 and
author author 		 t
author title 		 .
author title 		 moto
author title 		 -
author title 		 oka
author title 		 .
title title 		 application
title title 		 of
title title 		 hash
title title 		 to
title title 		 data
title title 		 base
title title 		 machine
title title 		 and
title title 		 its
title title 		 architecture
title booktitle 		 .
journal booktitle 		 new
journal booktitle 		 generation
journal booktitle 		 computing
journal booktitle 		 ,
volume booktitle 		 1
volume pages 		 (
volume booktitle 		 1
volume pages 		 )
volume PAD 		 ,
date booktitle 		 1983
date PAD 		 .


In [57]:
print_sentence_pred_label(xtest[1], ypred_test[1], ytest[1])

editor author 		 in
editor author 		 bouma
editor author 		 ,
editor author 		 h
editor author 		 .
editor author 		 ,
editor author 		 &
editor author 		 elsendoorn
editor author 		 ,
editor author 		 a
editor author 		 .
editor author 		 g
editor title 		 .
editor title 		 (
editor title 		 eds
editor title 		 .
editor title 		 )
editor title 		 ,
title title 		 working
title title 		 models
title title 		 of
title title 		 human
title title 		 perception
title title 		 ,
pages booktitle 		 pp
pages booktitle 		 .
pages booktitle 		 391
pages booktitle 		 -
pages booktitle 		 410
pages booktitle 		 .
publisher booktitle 		 academic
publisher pages 		 press
publisher pages 		 ,
location volume 		 london
location PAD 		 ,
location PAD 		 england
location PAD 		 .


**Model 2**:  Stacked Lstm

Accuracy : 43.4%

In [58]:
output_size = len(label_dic)
max_length_output = 118
class CitationNetwork(tf.keras.Model):
  def __init__(self, network_type = 'SimpleLSTM'):
      super(CitationNetwork, self).__init__()

      # self.optimizer = tf.keras.optimizers.RMSprop(1e-3)
      self.optimizer = tf.keras.optimizers.Adam(1e-3)
      self.embedding = layers.Embedding(input_dim=vocabulary_size, output_dim=embedding_size)
      
      # self.forward_layer1 = layers.LSTM(32, return_sequences=True, return_state=True)
      # self.backward_layer1 = layers.LSTM(32, return_sequences=True,return_state=True,go_backwards=True)
      # self.rnn1 = layers.Bidirectional(self.forward_layer1,merge_mode="concat",backward_layer = self.backward_layer1)

      # self.forward_layer = layers.LSTM(32, return_sequences=True, return_state=True)
      # self.backward_layer = layers.LSTM(32, return_sequences=True,return_state=True,go_backwards=True)
      # self.rnn = layers.Bidirectional(self.forward_layer,merge_mode="concat",backward_layer = self.backward_layer)

      self.rnn = layers.LSTM(256,return_sequences=True, return_state=True)
      self.rnn1 = layers.LSTM(256,return_sequences=True, return_state=True)
      self.rnn2 = layers.LSTM(256,return_sequences=True, return_state=True)
     

      self.fc = layers.Dense(output_size)
      
 
  def call(self, x, predict=True):
      emb = self.embedding(x)
      output1,state_h1,state_c1 = self.rnn(emb)
      output2,state_h2,state_c2 = self.rnn1(emb, initial_state = [state_h1,state_c1])
      output,state_h3,state_c3 = self.rnn2(emb,initial_state = [state_h2,state_c2])

      output = self.fc(output)
      if predict:
        return tf.math.argmax(output, axis=-1)
      return output
  
  def get_loss(self, ylogits, yt):
    # y - one hot vector which makes it easy to compute loss - KC
    ylabels = tf.one_hot(ybatch, output_size)
    cross_ent = tf.nn.softmax_cross_entropy_with_logits(logits=ylogits, labels=ylabels);
    return tf.reduce_sum(cross_ent)

@tf.function
def train_step(xbatch, ybatch):
    loss = 0
    with tf.GradientTape() as tape:
        prediction = cite(xbatch, False)
        # prediction = cite(xbatch, False)
        loss = cite.get_loss(prediction, ybatch)
        
    gradients = tape.gradient(loss, cite.trainable_variables)
    cite.optimizer.apply_gradients(zip(gradients, cite.trainable_variables))
    return loss

cite = CitationNetwork();

        

In [59]:
train_size = xdata.shape[0]
batch_size = 10
train_dataset = (tf.data.Dataset.from_tensor_slices(np.hstack((xdata, ydata)))
                 .shuffle(train_size).batch(batch_size))
# print(xdata.shape)
num_epoch = 1000
i = 0;
best_val = 0
test_val = 0

while i < num_epoch:
  loss = 0;
  for batch in train_dataset:
    xbatch = batch[:, :max_length_output]
    ybatch = batch[:, max_length_output:]
    loss += train_step(xbatch, ybatch)

  ypred_test = cite(xtest, predict=True).numpy()
  ypred_val = cite(xval, predict=True).numpy()
  ypred_train = cite(xdata, predict=True).numpy()
  print(i, loss.numpy(), perf(ypred_train, ypred_val, ypred_test, ydata, yval, ytest) )
  i = i+1
  # break

0 41848.746 Train: 0.397 Val: 0.397 Test: 0.434 -- Best Val: 0.397 Test: 0.434
1 22891.1 Train: 0.378 Val: 0.381 Test: 0.410 -- Best Val: 0.397 Test: 0.434
2 20399.193 Train: 0.376 Val: 0.376 Test: 0.401 -- Best Val: 0.397 Test: 0.434
3 19073.816 Train: 0.378 Val: 0.382 Test: 0.412 -- Best Val: 0.397 Test: 0.434
4 18485.547 Train: 0.384 Val: 0.390 Test: 0.422 -- Best Val: 0.397 Test: 0.434
5 18211.223 Train: 0.385 Val: 0.391 Test: 0.424 -- Best Val: 0.397 Test: 0.434
6 18045.51 Train: 0.383 Val: 0.389 Test: 0.420 -- Best Val: 0.397 Test: 0.434
7 17957.277 Train: 0.383 Val: 0.389 Test: 0.420 -- Best Val: 0.397 Test: 0.434
8 17885.637 Train: 0.384 Val: 0.389 Test: 0.421 -- Best Val: 0.397 Test: 0.434
9 17822.941 Train: 0.383 Val: 0.389 Test: 0.421 -- Best Val: 0.397 Test: 0.434
10 17758.809 Train: 0.384 Val: 0.389 Test: 0.421 -- Best Val: 0.397 Test: 0.434
11 17734.717 Train: 0.384 Val: 0.389 Test: 0.421 -- Best Val: 0.397 Test: 0.434
12 17680.438 Train: 0.384 Val: 0.389 Test: 0.420 -- B

KeyboardInterrupt: ignored